# 11.5 时期及其算术运算

时期（period）表示的是时间区间，比如数日、数月、数季、数年等。Period类所表示的就是这种数据类型，其构造函数需要用到一个字符串或整数，以及表11-4中的频率：

In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

In [3]:
p = pd.Period(2007, freq='A-DEC')

p

Period('2007', 'A-DEC')

这里，这个Period对象表示的是从2007年1月1日到2007年12月31日之间的整段时间。只需对Period对象加上或减去一个整数即可达到根据其频率进行位移的效果：

In [4]:
p + 5

Period('2012', 'A-DEC')

In [5]:
p - 2

Period('2005', 'A-DEC')

如果两个Period对象拥有相同的频率，则它们的差就是它们之间的单位数量：

In [6]:
pd.Period('2014', freq='A-DEC') - p

<7 * YearEnds: month=12>

period_range函数可用于创建规则的时期范围：

In [8]:
rng = pd.period_range('2000-01-01', '2000-06-30', freq='M')

rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]')

PeriodIndex类保存了一组Period，它可以在任何pandas数据结构中被用作轴索引：

In [10]:
pd.Series(np.random.randn(6), 
          index=rng
         )

2000-01   -1.434144
2000-02    0.739820
2000-03    1.215370
2000-04   -0.557866
2000-05    0.811936
2000-06    1.017217
Freq: M, dtype: float64

如果你有一个字符串数组，你也可以使用PeriodIndex类：

In [11]:
values = ['2001Q3', '2002Q2', '2003Q1']

index = pd.PeriodIndex(values, freq='Q-DEC')

index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]')

## 时期的频率转换

Period和PeriodIndex对象都可以通过其asfreq方法被转换成别的频率。假设我们有一个年度时期，希望将其转换为当年年初或年末的一个月度时期。该任务非常简单：

In [12]:
p = pd.Period('2007', freq='A-DEC')

p

Period('2007', 'A-DEC')

In [13]:
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [14]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

你可以将Period('2007','A-DEC')看做一个被划分为多个月度时期的时间段中的游标。图11-1对此进行了说明。对于一个不以12月结束的财政年度，月度子时期的归属情况就不一样了：

In [16]:
p = pd.Period('2007', freq='A-JUN')

p

Period('2007', 'A-JUN')

In [20]:
print('start: ', p.asfreq('M', 'start'))
print('end:   ', p.asfreq('M', 'end'))

start:  2006-07
end:    2007-06


在将高频率转换为低频率时，超时期（superperiod）是由子时期（subperiod）所属的位置决定的。例如，在A-JUN频率中，月份“2007年8月”实际上是属于周期“2008年”的：

In [22]:
p = pd.Period('Aug-2007', 'M')


p.asfreq('A-JUN')

Period('2008', 'A-JUN')

完整的PeriodIndex或TimeSeries的频率转换方式也是如此：

In [28]:
rng = pd.period_range('2006', '2009', freq='A-DEC')

rng

PeriodIndex(['2006', '2007', '2008', '2009'], dtype='period[A-DEC]')

In [26]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

ts

2006    0.921310
2007    0.760378
2008   -1.307403
2009   -2.737970
Freq: A-DEC, dtype: float64

In [27]:
ts.asfreq('M', how='start')

2006-01    0.921310
2007-01    0.760378
2008-01   -1.307403
2009-01   -2.737970
Freq: M, dtype: float64

这里，根据年度时期的第一个月，每年的时期被取代为每月的时期。如果我们想要每年的最后一个工作日，我们可以使用“B”频率，并指明想要该时期的末尾：

In [29]:
ts.asfreq('B', how='end')

2006-12-29    0.921310
2007-12-31    0.760378
2008-12-31   -1.307403
2009-12-31   -2.737970
Freq: B, dtype: float64

## 按季度计算的时期频率

季度型数据在会计、金融等领域中很常见。许多季度型数据都会涉及“财年末”的概念，通常是一年12个月中某月的最后一个日历日或工作日。就这一点来说，时期"2012Q4"根据财年末的不同会有不同的含义。pandas支持12种可能的季度型频率，即Q-JAN到Q-DEC：

In [31]:
p = pd.Period('2012Q4', freq='Q-JAN')

p

Period('2012Q4', 'Q-JAN')

在以1月结束的财年中，2012Q4是从11月到1月（将其转换为日型频率就明白了）。图11-2对此进行了说明：

In [32]:
p.asfreq('D', 'start')

Period('2011-11-01', 'D')

In [33]:
p.asfreq('D', 'end')

Period('2012-01-31', 'D')

因此，Period之间的算术运算会非常简单。例如，要获取该季度倒数第二个工作日下午4点的时间戳，你可以这样：

In [34]:
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60


p4pm

Period('2012-01-30 16:00', 'T')

In [35]:
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

period_range可用于生成季度型范围。季度型范围的算术运算也跟上面是一样的：

In [38]:
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')

ts = pd.Series(np.arange(len(rng)), index=rng)

ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64

In [39]:
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60

In [41]:
ts.index = new_rng.to_timestamp()

ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int64

## 将Timestamp转换为Period（及其反向过程）

通过使用to_period方法，可以将由时间戳索引的Series和DataFrame对象转换为以时期索引：

In [43]:
rng = pd.date_range('2000-01-01', periods=3, freq='M')

ts = pd.Series(np.random.randn(3), index=rng)

ts

2000-01-31    0.212477
2000-02-29    0.138485
2000-03-31    0.971127
Freq: M, dtype: float64

In [45]:
pts = ts.to_period()

pts

2000-01    0.212477
2000-02    0.138485
2000-03    0.971127
Freq: M, dtype: float64

由于时期指的是非重叠时间区间，因此对于给定的频率，一个时间戳只能属于一个时期。新PeriodIndex的频率默认是从时间戳推断而来的，你也可以指定任何别的频率。结果中允许存在重复时期：

In [48]:
rng = pd.date_range('1/29/2000', periods=6, freq='D')

ts2 = pd.Series(np.random.randn(6), index=rng)

ts2

2000-01-29   -0.764474
2000-01-30   -0.224911
2000-01-31   -0.776869
2000-02-01   -0.380102
2000-02-02    0.788719
2000-02-03    0.360758
Freq: D, dtype: float64

In [49]:
ts2.to_period('M')

2000-01   -0.764474
2000-01   -0.224911
2000-01   -0.776869
2000-02   -0.380102
2000-02    0.788719
2000-02    0.360758
Freq: M, dtype: float64

要转换回时间戳，使用to_timestamp即可：

In [50]:
pts = ts2.to_period()

pts

2000-01-29   -0.764474
2000-01-30   -0.224911
2000-01-31   -0.776869
2000-02-01   -0.380102
2000-02-02    0.788719
2000-02-03    0.360758
Freq: D, dtype: float64

In [51]:
pts.to_timestamp(how='end')

2000-01-29 23:59:59.999999999   -0.764474
2000-01-30 23:59:59.999999999   -0.224911
2000-01-31 23:59:59.999999999   -0.776869
2000-02-01 23:59:59.999999999   -0.380102
2000-02-02 23:59:59.999999999    0.788719
2000-02-03 23:59:59.999999999    0.360758
Freq: D, dtype: float64

## 通过数组创建PeriodIndex

固定频率的数据集通常会将时间信息分开存放在多个列中。例如，在下面这个宏观经济数据集中，年度和季度就分别存放在不同的列中：

In [54]:
data = pd.read_csv('../../examples/macrodata.csv')

data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [55]:
data.year

0      1959
1      1959
2      1959
3      1959
4      1960
       ... 
198    2008
199    2008
200    2009
201    2009
202    2009
Name: year, Length: 203, dtype: int64

In [56]:
data.quarter

0      1
1      2
2      3
3      4
4      1
      ..
198    3
199    4
200    1
201    2
202    3
Name: quarter, Length: 203, dtype: int64

通过将这些数组以及一个频率传入PeriodIndex，就可以将它们合并成DataFrame的一个索引：

In [57]:
index = pd.PeriodIndex(year=data.year, 
                       quarter=data.quarter,
                       freq='Q-DEC'
                      )

index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203)

In [59]:
data.index = index

data.infl

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
          ... 
2008Q3   -3.16
2008Q4   -8.79
2009Q1    0.94
2009Q2    3.37
2009Q3    3.56
Freq: Q-DEC, Name: infl, Length: 203, dtype: float64